In [1]:
#ch12_ALS_01

In [2]:
#1. 準備資料

In [3]:
#step1.1  匯入 ml-100k資料, 使用sc.textFile() ,Transformation

In [4]:
rawUserData=sc.textFile("u.data") #rawUserData 是一個 RDD

In [5]:
rawUserData.count() #count action, 100k 評價資料

100000

In [6]:
#step1.2  查看首筆資料, first() action, byte , '\t' -->tab 間隔, user_id, product_id, rating, timestamp
rawUserData.first()

u'196\t242\t3\t881250949'

In [7]:
print(rawUserData.first()) #用print() 

196	242	3	881250949


In [8]:
#step1.3 Import Rating function, (from pyspark.mllib.recommendation module)
#             Represents a (user, product, rating) tuple.
from pyspark.mllib.recommendation import Rating

In [9]:
#step1.4 讀取RDD rawUserData 前三個欄位[:3], 記住原始資料是以 '\t' 分隔
#              將RDD rawUserData --> RDD rawRatings
rawRatings=rawUserData.map(lambda x:x.split("\t")[:3]) 

In [10]:
#step1.5 檢視 RDD rawRatings 前5列, take() action
#              rawRatings 是 PiplelinedRDD 物件(形式是 list of list)
rawRatings.take(5)

[[u'196', u'242', u'3'],
 [u'186', u'302', u'3'],
 [u'22', u'377', u'1'],
 [u'244', u'51', u'2'],
 [u'166', u'346', u'1']]

In [11]:
#step1.6 準備ALS訓練資料
#將rawRating資料(lPiplelinedRDD) 以map() 轉換成 PipelinedRDD ratingsRDD
#   ratingsRDD 是ALS演算法所需的資料格式
ratingsRDD=rawRatings.map(lambda x:(x[0],x[1],x[2]))  #形式是  tuple (x[0],x[1],x[2])

In [12]:
#step1.7 檢視 ratingsRDD 前5筆 , take(5) action
#ratingsRDD -->形式是 list of tuple
ratingsRDD.take(5)

[(u'196', u'242', u'3'),
 (u'186', u'302', u'3'),
 (u'22', u'377', u'1'),
 (u'244', u'51', u'2'),
 (u'166', u'346', u'1')]

In [13]:
#step1.8 檢視ratingsRDD 筆數
ratingsRDD.count()

100000

In [14]:
#step1.9 檢視不重複使用者數/電影數

In [15]:
numUsers=ratingsRDD.map(lambda x:x[0]).distinct().count()
numMovies=ratingsRDD.map(lambda x:x[1]).distinct().count()
print('不重複使用者數: '+str(numUsers)+'   不重複電影數:'+str(numMovies))

不重複使用者數: 943   不重複電影數:1682


In [22]:
#2. 訓練模型

In [23]:
#step2.1 Import ALS module                          

In [24]:
from pyspark.mllib.recommendation import ALS

In [25]:
#step2.2 使用ALS.train() method 進行訓練

In [26]:
#參數: Ratings=ratingsRDD, rank=10, Iterations=10, lambda=0.01
#回傳資料型別: MatrixFactorizationModel 
#明確評價訓練
model=ALS.train(ratingsRDD,10,10,0.01)

In [27]:
type(model) #訓練完成, 結果是 MatrixFactorizationModel Object 'model'

pyspark.mllib.recommendation.MatrixFactorizationModel

In [28]:
#3. 使用模型進行推薦

In [29]:
#step3.1 使用 MatrixFactorizationModel.recommendProducts(user:int,num:int) method
#             針對使用者推薦電影

In [30]:
model.recommendProducts(100,5) #userid=100, num=5 

[Rating(user=100, product=263, rating=6.133248850324193),
 Rating(user=100, product=1252, rating=5.800824652275878),
 Rating(user=100, product=867, rating=5.7776484985911125),
 Rating(user=100, product=718, rating=5.766984639488829),
 Rating(user=100, product=904, rating=5.606909488206091)]

In [31]:
#rating 越高,表示越推薦

In [32]:
#step3.2 查詢系統對使用者(userid)推薦的產品(products) 的評分(rating)
#             使用 MatrixFactorizationModel.predict(user:int,product:int) method

In [33]:
model.predict(100,1141) #userid=100, product=1141, rating=

2.9462244768522146

In [34]:
#step3.3 針對電影推薦給使用者
#             使用MatrixFactorizationModel.recommendUsers(product:int,num:int) method

In [35]:
model.recommendUsers(200,5) #product=200, num=5

[Rating(user=672, product=200, rating=6.480740249127878),
 Rating(user=732, product=200, rating=6.408603081054213),
 Rating(user=820, product=200, rating=6.364824303481043),
 Rating(user=811, product=200, rating=6.01147453924953),
 Rating(user=252, product=200, rating=5.940072296381982)]

In [36]:
#4. 顯示推薦的電影的名稱

In [37]:
#step4.1 讀取u.item 文字檔
#              使用sc.textFile() Transformation
#              將 u.item 文字檔匯入 itemRDD

In [38]:
itemRDD=sc.textFile('u.item')
itemRDD.count()

1682

In [39]:
#step4.2 建立 電影ID與名稱 字典

In [40]:
itemRDD.take(5) #資料是以 "|" 分隔

[u'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0',
 u'2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 u'3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 u'4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 u'5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0']

In [41]:
movieTitle=itemRDD.map(lambda x:x.split("|")).map(lambda a:(float(a[0]),a[1])).collectAsMap()

In [42]:
type(movieTitle) #collectAsMap() 回傳的是 dict object 'moviewTitle'

dict

In [50]:
#step4.3 顯示字典前5筆 

In [51]:
movieTitle.items()[:5]

[(1.0, u'Toy Story (1995)'),
 (2.0, u'GoldenEye (1995)'),
 (3.0, u'Four Rooms (1995)'),
 (4.0, u'Get Shorty (1995)'),
 (5.0, u'Copycat (1995)')]

In [52]:
#step4.4 查詢電影名稱

In [53]:
movieTitle[5.0] #movieID 5的電影名稱

u'Copycat (1995)'

In [54]:
#step4.5 顯示前5筆推薦的電影名稱

In [55]:
recommendP=model.recommendProducts(100,5)

In [77]:
for p in recommendP:
    print("對會員:"+str(p[0])+" 推薦電影:")
    print(movieTitle[p[1]])
    print(" 推薦評分:"+str(p[2]))

對會員:100 推薦電影:
Steel (1997)
 推薦評分:6.13324885032
對會員:100 推薦電影:
Contempt (M�pris, Le) (1963)
 推薦評分:5.80082465228
對會員:100 推薦電影:
Whole Wide World, The (1996)
 推薦評分:5.77764849859
對會員:100 推薦電影:
In the Bleak Midwinter (1995)
 推薦評分:5.76698463949
對會員:100 推薦電影:
Ma vie en rose (My Life in Pink) (1997)
 推薦評分:5.60690948821
